import libraries

In [10]:
import tensorflow as tf
import argparse
from datetime import datetime
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

from data_loader import DataGenerator

from trainer import Trainer
from preprocessor import DynamicPreprocessor
from utils.dirs import create_dirs
from utils.logger import Logger

now = datetime.now()

let's make data

In [11]:
train_size = 20000
val_size = 2000
base_dir = "/home/angrypark/sandbox/data/matching_model/v0/sol_selected_1.txt"
batch_size = 128

In [12]:
queries = list()
replies = list()
val_queries = list()
val_replies = list()
with open("/home/angrypark/sandbox/data/matching_model/v0/sol_selected_1.txt", "r") as f:
    for _ in range(train_size):
        line = f.readline()
        splits = line.split("\t")
        queries.append(splits[0])
        replies.append(splits[1].strip())
    for _ in range(val_size):
        line = f.readline()
        splits = line.split("\t")
        val_queries.append(splits[0])
        val_replies.append(splits[1].strip())

 26%|██▌       | 200/782 [00:20<00:58, 10.00it/s]

In [14]:
from sampler import RandomSampler
sampler = RandomSampler(config)
val_sampler = RandomSampler(config)

In [15]:
sampler.fit(replies)
val_sampler.fit(val_replies)

write train, val text data

In [16]:
%%time
sampled_queries, sampled_replies, labels = sampler.sample(queries, replies, add_echo=False)

CPU times: user 38.4 s, sys: 120 ms, total: 38.5 s
Wall time: 37.9 s


In [17]:
with open("data/train_no_echo.txt", "w") as f:
    for q, r, l in zip(sampled_queries, sampled_replies, labels):
        f.write(q + "\t" + r + "\t" + str(l) + "\n")

In [16]:
%%time
val_sampled_queries, val_sampled_replies, val_labels = val_sampler.sample(val_queries, val_replies)

CPU times: user 480 ms, sys: 8 ms, total: 488 ms
Wall time: 477 ms


In [17]:
with open("data/val_no_echo.txt", "w") as f:
    for q, r, l in zip(val_sampled_queries, val_sampled_replies, val_labels):
        f.write(q + "\t" + r + "\t" + str(l) + "\n")

모델에 돌려보자

In [1]:
import tensorflow as tf
import argparse
from datetime import datetime
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

from data_loader import DataGenerator

from trainer import Trainer
from preprocessor import DynamicPreprocessor
from utils.dirs import create_dirs
from utils.logger import Logger

now = datetime.now()

In [2]:
class Config:
    def __init__(self):
        self.name = "debug"
        self.mode = "train"
        self.config = ""
        self.train_dir = "data/train_no_echo.txt"
        self.val_dir = "data/val_no_echo.txt"
        self.pretrained_embed_dir = "/home/shuuki4/sandbox_project/fasttext/results/ft_256.bin"
        self.model = "DualEncoderLSTM"
        self.normalizer = "DummyNormalizer"
        self.tokenizer = "DummyTokenizer"
        self.vocab_size = 20000
        self.learning_rate = 1e-5
        self.min_length = 1
        self.max_length = 15
        self.lstm_dim = 512 # need to check
        self.batch_size = 256
        self.num_epochs = 200
        self.evaluate_every = 1
        self.checkpoint_every = 1
        self.max_to_keep = 10
        self.shuffle = True
        self.gpu = 1
        self.embed_dim = 256

config = Config()

In [3]:
device_config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)
device_config.gpu_options.per_process_gpu_memory_fraction = 0.2

In [4]:
sess = tf.Session(config=device_config)

In [5]:
preprocessor = DynamicPreprocessor(config)
data = DataGenerator(preprocessor, config)

 24%|██▍       | 95345/400000 [00:00<00:00, 953250.91it/s]

Build Vocabulary...


100%|██████████| 400000/400000 [00:00<00:00, 963868.25it/s]


Vocabulary Built. Total number of unique tokens :  19766


In [6]:
from models.dual_encoder_lstm import DualEncoderLSTM
model = DualEncoderLSTM(preprocessor, config)

Loading pre-trained embedding from /home/shuuki4/sandbox_project/fasttext/results/ft_256.bin...
Embedding loaded, number of OOV : 0 / 19770


In [7]:
logger = Logger(sess, config)

In [8]:
trainer = Trainer(sess, model, data, config, logger)

In [9]:
trainer.train()

100%|██████████| 782/782 [00:23<00:00, 33.54it/s]


Epoch :  1 | Train loss : 0.0876 | Train accuracy : 0.8945 | Val loss : 0.6467 | Val accuracy : 0.9000 


  1%|          | 4/782 [00:00<00:22, 34.20it/s]

Model saved


100%|██████████| 782/782 [00:23<00:00, 33.79it/s]


...

  1%|          | 4/782 [00:00<00:23, 32.50it/s]

Model saved


 27%|██▋       | 208/782 [00:06<00:16, 33.79it/s]

KeyboardInterrupt: 

 27%|██▋       | 208/782 [00:20<00:55, 10.39it/s]